### Installing Packages

In [35]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

#### Create Fictional Data

In [6]:
Movie_Ratings = {'Rating_1' : [0,0,1,1,1],
                 'Rating_2' : [1,1,0,0,1],
                 'My_Ratings' : [0,0,1,1,1]}

movie_df = pd.DataFrame(Movie_Ratings)
initial_weights = [0.5,0.5]
initail_intercept = 3.0

##### Creating a Function to calculate z-values

In [16]:
def get_z_vals(x, weights_vector, intercept):
    z = []
    for i in range(len(x)):
        line = []
        for j in range(x.shape[1]):
            line.append(x.iloc[i,:][j] * weights_vector[j])
        z.append(sum(line) + intercept)
    return z

In [17]:
get_z_vals(x_test, initial_weights, initail_intercept)

[3.5, 3.5, 3.5, 3.5, 4.0]

##### $y_{pred}$ function

In [18]:
def get_y_hat(z_vector):
    y_hat = []
    for z_score in z_vector:
        y_hat.append(1/(1+math.exp(-z_score)))
    
    return y_hat

In [19]:
get_y_hat(get_z_vals(x_test, initial_weights, initail_intercept))

[0.9706877692486436,
 0.9706877692486436,
 0.9706877692486436,
 0.9706877692486436,
 0.9820137900379085]

#### Cost function

In [20]:
def cost_fun(y, y_hat):    
    return -1/len(y_hat) * sum([y.iloc[i]*math.log(y_hat[i]) + 
                     (1-y.iloc[i])* math.log(1-(y_hat[i]))for i in range(len(y_hat))])[0]

In [21]:
cost_fun(y_test, get_y_hat(get_z_vals(x_test, initial_weights, initail_intercept)))

1.427430320201658

##### Updating Parameters

In [22]:
def update_intercept(y, y_hat, alpha, intercept):
    
    difference = [y_hat[i] - y.iloc[i] for i in range(len(y_hat))]
    
    return (intercept - (alpha * (sum(difference)/len(difference))))[0]

In [23]:
update_intercept(y_test, get_y_hat(get_z_vals(x_test, initial_weights, initail_intercept)), 0.01, initail_intercept)

2.996270470265935

In [24]:
def update_weight_vector(y, y_hat, alpha, weight_vec, x):
    
    difference = [y_hat[i] - y.iloc[i] for i in range(len(y_hat))]
    
    del_w = []
    for i in range(len(weight_vec)):
        delta_w = sum([difference[j] * x.iloc[j][i] for j in range(len(difference))]) / len(difference)
        del_w.append(delta_w)
        
    return [(weight_vec[i] - (alpha) * del_w[i])[0] for i in range(len(weight_vec))]


In [25]:
update_weight_vector(y_test, get_y_hat(get_z_vals(x_test, initial_weights, initail_intercept)),
                     0.01, initial_weights, x_test)

[0.5001532213429296, 0.4961532213429296]

### Testing

In [26]:
x_test = pd.DataFrame({'Rating_1' : [0,0,1,1,1],
                 'Rating_2' : [1,1,0,0,1]})
y_test = pd.DataFrame({'My_Ratings' : [0,0,1,1,1]})
initial_weights = [0.5,0.5]
initail_intercept = 3.0

In [85]:
def gradient_descent(x,y, init_w, init_b, alpha = 0.01):
    
    #Get z values using x, init_w and init b
    z = get_z_vals(x, init_w, init_b)
        
    #Get y prediction values
    y_hat = get_y_hat(z)
        
    # Calc Cost Function 
    Cost_Fun = cost_fun(y, y_hat)
    
    #Delta w and Delta b
    intercept_new = update_intercept(y, y_hat, alpha, init_b)
        
    new_weights = update_weight_vector(y, y_hat, alpha, init_w, x)

    
    return new_weights, Cost_Fun, intercept_new, [round(pred) for pred in y_hat]

In [86]:
gradient_descent(x_test,y_test, initial_weights, initail_intercept)

([0.5001532213429296, 0.4961532213429296],
 1.427430320201658,
 2.996270470265935,
 [1, 1, 1, 1, 1])

##### Creting Iterative Function to handle iterations/tolerance

In [124]:
def grad_its(x,y, init_w, init_b, alpha = 0.01, tol = 1e-8, max_its = 1000):
    i = 0
    new_weights, Cost_Fun, intercept_new, _ = gradient_descent(x,y, init_w, init_b, alpha)
    new_weights, Cost_Fun_new, intercept_new , _= gradient_descent(x,y,new_weights, intercept_new, alpha)
    
    while (abs(Cost_Fun - Cost_Fun_new) > tol) and i < max_its:
        Cost_Fun = Cost_Fun_new
        new_weights, Cost_Fun_new, intercept_new, y_hat = gradient_descent(x,y,new_weights, intercept_new, alpha)
        i += 1 
        print(i)
    
    return new_weights, Cost_Fun_new, intercept_new, y_hat

In [102]:
grad_its(x_test,y_test, [1.6533184539468795, -1.9471455873302386], 1.0005869287065845,alpha = 0.1, max_its = 1000)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


([5.331811048405806, -3.2087120814758907],
 0.03975903139799564,
 0.3190871459055846)

In [92]:
blobs = pd.read_csv("/Users/joshocurry/Downloads/blobs300.csv")
blobs

,X1,X2,X3,X4,Class
0,0.726739,0.035393,1.759229,1.633698,1
1,2.691834,2.715685,-0.085346,2.131453,0
2,2.446502,3.853792,0.487143,2.138328,0
3,0.899178,-0.596257,1.764085,1.863523,1
4,0.520531,0.521058,1.607952,1.757980,1
...,...,...,...,...,...
295,1.324187,0.125884,1.759156,2.089274,1
296,2.095409,3.344137,0.725646,2.181084,0
297,1.524761,3.281810,-0.855361,2.572439,0
298,2.597079,3.353805,1.101402,1.850293,0


In [93]:
x_blobs = blobs[['X1', 'X2', 'X3', 'X4']]
y_blobs = blobs[['Class']]

In [204]:
grad_its(x_blobs,y_blobs, [1,1,1,1] , 10)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


([-1.2974544127210041,
  -2.8621849481304396,
  0.9455682174016076,
  -0.6800520240761868],
 0.01699155266255347,
 8.901394350229799)

In [229]:
blob_opt = [-1.2974544127210041,
  -2.8621849481304396,
  0.9455682174016076,
  -0.6800520240761868]
b_opt = 8.901394350229799

z = []
for i in range(len(x_blobs)):
    line = []
    for j in range(x_blobs.shape[1]):
        line.append(x_blobs.iloc[i,:][j] * blob_opt[j])
    z.append(sum(line) + b_opt)

#Get y prediction values
y_hat = []
for z_score in z:
    y_hat.append(1/(1+math.exp(-z_score)))
       
errors = []
for pair in list(zip(y_blobs['Class'],y_hat)):
    errors.append(abs(pair[0] - pair[1]))
    
print(sum(errors)/len(errors))
print(max(errors))

0.015672735914413042
0.3633951503524396


In [319]:
get_z_vals_vals_vals(x_blobs, blob_opt, b_opt)

[8.409654994810595,
 -3.894129329265816,
 -6.296642332066293,
 9.842119672781015,
 7.059573962562666,
 -5.619838605008898,
 -4.0811260089403,
 -4.946853754893775,
 -3.1440558554347593,
 7.189378065863846,
 5.717266598611658,
 -4.333328342224709,
 7.576732792811905,
 -5.612699687757074,
 -4.580713210277285,
 6.15616065416236,
 8.699226724864975,
 5.579858364004253,
 -0.6725333871933437,
 7.585672156969465,
 -5.942487824514965,
 5.434581989608764,
 -8.232474445564831,
 -6.503819782485046,
 -5.764964246537911,
 -4.492422474325576,
 -9.048827358837377,
 -5.368550024128739,
 7.895318451530372,
 -4.033791681448767,
 -4.7253089445773195,
 -5.096346838139835,
 -4.855175455210729,
 7.1183952087127595,
 -1.8656301427227788,
 -5.545179157860668,
 -3.86954697218218,
 -3.7119897887704276,
 6.580507687823696,
 -5.675514910978253,
 7.227957719360623,
 -7.591176411404456,
 7.225303082973934,
 8.32921727560763,
 4.987947541292299,
 7.296171887846249,
 -4.715263165264492,
 -5.030895527022237,
 -3.167457

In [33]:
pairs = list(zip(y_blobs['Class'],[round(val) for val in y_hat]))

print(sum([1 for pair in pairs if pair[0] == pair[1]])/len(pairs))

NameError: name 'y_blobs' is not defined

In [96]:
circles = pd.read_csv("/Users/joshocurry/Downloads/circles600.csv")
circles

,X1,X2,Class
0,-0.371514,0.012406,1
1,-1.014230,0.223095,0
2,-1.123804,0.249026,0
3,0.213708,0.032912,1
4,-0.833531,-0.491312,0
...,...,...,...
595,-0.100827,-0.386171,1
596,0.533986,-0.144301,1
597,-0.445042,-0.042955,1
598,-0.052254,-0.324085,1


In [97]:
x_circles = circles[['X1', 'X2']]
y_circles = circles[['Class']]

In [98]:
grad_its(x_circles,y_circles, [1,1] , 10)

ValueError: too many values to unpack (expected 3)

In [320]:
circles_pred = get_y_hat(get_z_vals(x_circles, [1.0146696399758612, 1.056396127381332], 5.007848840155507))

In [321]:
pairs = list(zip(y_circles['Class'],[round(val) for val in circles_pred]))

print(sum([1 for pair in pairs if pair[0] == pair[1]])/len(pairs))

0.5


In [135]:
np.random.normal(loc = 0, scale = 0.1)

-0.0940676702726047

In [182]:
def one_hidden_layer(x,y,n_hidden_nodes = 4,alpha = 0.01, tol = 1e-8, max_its = 1000):
    
    hidden_dict = dict()
    
    test = []
    
    for i in range(n_hidden_nodes):
        
        init_w = list(abs(np.random.normal(loc = 0, scale = 0.1, size = (1,x.shape[1])))[0]) 
        
        *_,predictions = grad_its(x,y,
                         init_w,
                         np.random.normal(loc = 0, scale = 0.1)
                         ,alpha, tol, max_its)
        hidden_dict[i] = predictions
        test.append(init_w)
        
    hidden_df = pd.DataFrame(hidden_dict)
    
    final_weights = list(abs(np.random.normal(loc = 0, scale = 0.1, size = (1,n_hidden_nodes)))[0]) 
    print('FINAL +++++++++++++++++++++++++++++++++')
    return  grad_its(hidden_df,y,
                         final_weights,
                         np.random.normal(loc = 0, scale = 0.1))[-1]
    

In [183]:
one_hidden_layer(x_blobs, y_blobs,alpha = 0.1, tol = 1e-5, max_its = 1000)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379


222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471


[1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,


In [184]:
circles_predictions = one_hidden_layer(x_circles, y_circles, 2, alpha = 0.1, tol = 1e-5, max_its = 1000)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
FINAL +++++++++++++++++++++++++++++++++
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166

In [196]:

total = list(zip(circles_predictions, y_circles.iloc[:,0]))
print(sum([1 for pair in total if pair[0] == pair[1]])/len(total))

0.6383333333333333


8